#### importando as bibliotecas


In [16]:
import os
from dotenv import load_dotenv, find_dotenv

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter

from langchain_cohere import CohereEmbeddings
from langchain_pinecone import PineconeVectorStore

from langchain.memory import ConversationBufferMemory

from langchain_groq import ChatGroq

from langchain import hub
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain

##### carregando as variáveis de ambiente

In [17]:
load_dotenv(find_dotenv())

COHERE_API_KEY = os.getenv("COHERE_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

#### carregando o arquivo em pdf


In [18]:
loader = PyPDFLoader("C:\\Users\\Maria Raquel\\Chatbot-with-pdf\\data\\2210.03629v3.pdf")
documents = loader.load()

#### fatiamento do conteudo

In [19]:
text_splitter = CharacterTextSplitter(separator="\n")
docs = text_splitter.split_documents(documents)

#### Gerando Embeddings (cohere) e Criando o Vetor com pinecone

In [20]:
embeddings = CohereEmbeddings(model="embed-english-v3.0", cohere_api_key=COHERE_API_KEY)
index_name = "rag-demo"

vectorstore_from_docs = PineconeVectorStore.from_documents(
    docs, index_name=index_name, embedding=embeddings
)

In [21]:
vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)
print(vectorstore.similarity_search("What is a llm?"))

[Document(id='14798b56-c6ff-43c7-945f-07ebf51b2593', metadata={'author': '', 'creationdate': '2023-03-13T00:09:11+00:00', 'creator': 'LaTeX with hyperref', 'keywords': '', 'moddate': '2023-03-13T00:09:11+00:00', 'page': 8.0, 'page_label': '9', 'producer': 'pdfTeX-1.40.21', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020) kpathsea version 6.3.2', 'source': 'C:\\Users\\Maria Raquel\\Chatbot-with-pdf\\data\\2210.03629v3.pdf', 'subject': '', 'title': '', 'total_pages': 33.0, 'trapped': '/False'}, page_content='Published as a conference paper at ICLR 2023\n5 R ELATED WORK\nLanguage model for reasoning Perhaps the most well-known work of using LLMs for reasoning\nis Chain-of-Thought (CoT) (Wei et al., 2022), which reveals the ability of LLMs to formulate their\nown “thinking procedure” for problem solving. Several follow-up works have since been performed,\nincluding least-to-most prompting for solving complicated tasks (Zhou et al., 2022), zero-shot-\nCoT (

#### criando a memoria pro chat

In [22]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

#### configurando a llm com o groq 

In [23]:
llm = ChatGroq(
    model="Gemma2-9b-It",
    groq_api_key=GROQ_API_KEY,
    temperature=0.1
)

#### cadeia de respostas e perguntas com RAG

In [ ]:
retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")
combine_docs_chain = create_stuff_documents_chain(
    llm, 
    retrieval_qa_chat_prompt
    )



c:\Users\Maria Raquel\Chatbot-with-pdf\venv\Lib\site-packages\langsmith\client.py:278: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [28]:
retrieval_chain = create_retrieval_chain(
    vectorstore.as_retriever(), 
    combine_docs_chain
)

#### perguntando ao modelo

In [29]:
response = retrieval_chain.invoke({"input": "What is ReAct in 3 sentences?"})
print(response['answer'])

ReAct is a method for training language models to perform complex reasoning tasks by prompting them to generate a sequence of thoughts and actions. 

It leverages the flexibility of language models to reason and act in an interactive manner, allowing them to retrieve information from external sources like Wikipedia and combine it with their internal knowledge. 

This approach results in interpretable and controllable decision-making processes, making it easier to understand and debug the reasoning behind the model's actions. 



